In [ ]:
라이브러리 임포트

In [1]:
import cv2 # 영상 크기변경 함수 사용
import numpy as np # 배열 처리 함수 사용 – reshape(), array(), save()
import os # 디렉토리 지정을 위해 임포트

from keras.utils import to_categorical # one-hot encoding
from random import shuffle # 네트워크가 잘 학습되도록 학습 데이터 섞음
from tqdm import tqdm # 작업에 대한 깔끔한 백분율 바 표시
import tensorflow as tf

TRAIN_DIR = './train' # 학습 영상 경로
TEST_DIR = './test' # 테스트 영상 경로
IMG_SIZE = 50             # 정규화 영상 크기
LR = 1e-6                 # 학습률(learning rate)- 경사하강 스텝
MODEL_NAME = 'dogsvscats-{}-{}.model'.format(LR, '2conv-basic')

Using TensorFlow backend.


In [ ]:
Keras에 있는 VGG-16을 이용한 개/고양이 분류 - 훈련 단계

In [21]:
# 모델 및 학습률 지정
def label_img(img):
    word_label = img.split('.')[-3] # 오른쪽에서 3번째 원소
    if word_label == 'cat': return 0 # [much cat, no dog]
    elif word_label == 'dog': return 1 # [no cat, very dog]

def create_train_data():
    training_data = []
    for img in tqdm(os.listdir(TRAIN_DIR)):
        label = label_img(img)
        path = os.path.join(TRAIN_DIR, img)
        img = cv2.imread(path)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE), interpolation=cv2.INTER_CUBIC)
        training_data.append([np.array(img),np.array(label)])
        
    shuffle(training_data)
    np.save('train_data.npy', training_data)
    return training_data

train_data = create_train_data()

100%|███████████████████████████████████████████████████████████████████████████| 25000/25000 [01:13<00:00, 342.03it/s]


In [15]:
from keras.applications.vgg16 import VGG16 # VGG16 함수 임포트
from keras.optimizers import Adam # Adam 최적화 함수 임포트

train_data = np.load('train_data.npy')   # 파일에서 학습데이터 읽기
train = train_data[:-5000] # 뒤 5000번째까지(0~19999)
test = train_data[-5000:] # 뒤 5000번째부터 마지막까지(2000~2499)

X_train = np.array([i[0] for i in train]).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
y_train = [i[1] for i in train]
y_train = to_categorical(y_train) # 학습 데이터 레이블 one-hot encoding
print(X_train.shape, y_train.shape)

X_test = np.array([i[0] for i in test]).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
y_test = [i[1] for i in test]
y_test = to_categorical(y_test) # 테스트 데이터 레이블 one-hot encoding

# 학습 모델 구축
model = VGG16(weights=None,                        # 초기가중치 없이 시작
              input_shape=(IMG_SIZE, IMG_SIZE, 3),  # 입력 데이터 형태
              classes=y_test.shape[1])              # 구분할 그룹 개수
 
# adam 최적화, 학습률 지정
model.compile(Adam(lr=LR), "categorical_crossentropy", metrics=["accuracy"])
model.summary() # 모델 정보 요약
model.fit(X_train, y_train, validation_data=(X_test, y_test), 
          epochs=20, batch_size=256, verbose=2) # 모델 학습 수행

# 모델 평가
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: {} \nError: {}".format(scores[1], 100-scores[1]*100))

model.save('VGG-16_keras1.h5')                 # 학습 모델 저장

(20000, 50, 50, 3) (20000, 2)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         (None, 50, 50, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 50, 50, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 50, 50, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 25, 25, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 25, 25, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 25, 25, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 12, 12, 12

In [ ]:
개 영상의 학습된 특징 시각화 - 64개 필터 사용 

In [3]:
from keras.models import Model
import matplotlib.pylab as pylab
from keras import models

train_data = np.load('train_data.npy')
train = train_data[:-5000]
X_train = np.array([i[0] for i in train]).reshape(-1,IMG_SIZE,IMG_SIZE,3)

model = models.load_model('VGG-16_keras1.h5')

outputs=model.get_layer('block1_conv2').output
intermediate_layer_model = Model(inputs=model.input, outputs=outputs)
intermediate_output = intermediate_layer_model.predict(X_train)

fig = pylab.figure(figsize=(10,10))
fig.subplots_adjust(left=0, right=1, bottom=0, top=1, hspace=0.05, wspace=0.05)
pylab.gray()

i = 3
for c in range(64):
    pylab.subplot(8, 8, c+1), pylab.axis('off')
    pylab.imshow(intermediate_output[i, :, :, c])
pylab.show()

InternalError: Could not allocate ndarray

In [ ]:
두 번째 컨볼루션 계층 시각화 - 128개 필터

In [7]:
# train_data = np.load('train_data.npy')
# train = train_data[:-5000]
# X_train = np.array([i[0] for i in train]).reshape(-1,IMG_SIZE,IMG_SIZE,3)
# model = models.load_model('VGG-16_keras.h5')

outputs=model.get_layer('block2_conv2').output
intermediate_layer_model = Model(inputs=model.input, outputs=outputs)
intermediate_output = intermediate_layer_model.predict(X_train)

fig = pylab.figure(figsize=(10,10))
fig.subplots_adjust(left=0, right=1, bottom=0, top=1, hspace=0.05, wspace=0.05)
pylab.gray()

i = 3
for c in range(128):
    pylab.subplot(10, 13, c+1), pylab.axis('off')
    pylab.imshow(intermediate_output[i, :, :, c])
pylab.show()

MemoryError: 

In [ ]:
Testing(예측) 단계

In [6]:
def process_test_data():
    testing_data = []
    for img in os.listdir(TEST_DIR):
        path = os.path.join(TEST_DIR, img)
        img_num = img.split('.')[0]
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        testing_data.append([np.array(img), img_num])
        
    np.save('test_data.npy', testing_data)
    return testing_data

# test_data = process_test_data()
# # test_data = np.load('test_data.npy')             # 저장된 파일에서 읽기
# len(test_data)

X_test = np.array([i for i in test_data]).reshape(-1,IMG_SIZE,IMG_SIZE,3)
probs = model.predict(X_test)
probs = np.round(probs,2)

pylab.figure(figsize=(20,20))
for i in range(100):
    pylab.subplot(10,10,i+1), pylab.imshow(X_test[i,:,:,::-1]), pylab.axis('off')
    result = 'cat' if probs[i][1] < 0.5 else 'dog'
    pylab.title("{}, prob={:0.2f}".format(result , max(probs[i][0],probs[i][1])))
pylab.show()

NameError: name 'test_data' is not defined